# Importing Libraries

In [26]:
import numpy as np
import pandas as pd

In [27]:
data = pd.read_csv("movies.csv")


In [28]:
data.shape

(9742, 3)

In [29]:
data.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [30]:
data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [31]:
data.tail()

,movieId,title,genres
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy


# Node class

In [32]:
class Node:
    def __init__(self, feature=None, threshold=None, value=None, var_red=None, left=None, right=None):
        
         # for decision node
        self.feature = feature      # feature to split on
        self.threshold = threshold  # threshold value to split on
        self.left = left            # left subtree
        self.right = right          # right subtree
        self.var_red = var_red      # variance reduction
        # for leaf node
        self.value = value          # predicted value 

# Tree Class 

In [33]:
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=3, max_depth=3):
        
     # initialize the root of the tree 
        self.root = None
        
     # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    # recursive function to build the tree
    def build_tree(self, dataset, curr_depth=0):       
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        
     # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            
     # find the best split
            best_split = self.find_best_split(dataset, num_samples, num_features)
            
            # check if SDR is positive or not
            if best_split["var_red"]>0:
                
            # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                
            # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                
            # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        
        # return leaf node
        return Node(value=leaf_value)
    
    

In [34]:
# To find the best split
def find_best_split(self, dataset, num_samples, num_features):
        
        # To store the best split
        best_split = {}
        max_var_red = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            
        # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                
                # check if childs are not null
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    
                # compute SDR
                    curr_var_red = self.variance_reduction(y, left_y, right_y)
                    
                    # update the best split if needed
                    if curr_var_red > max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
                        
        # return best split
        return best_split
    
    # splitting the data 
def split(self, dataset, feature_index, threshold):
    dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
    dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
    return dataset_left, dataset_right
    
    

In [35]:
# computing the variance reduction
def variance_reduction(self, parent, l_child, r_child):
    weight_l = len(l_child) / len(parent)
    weight_r = len(r_child) / len(parent)
    reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r * np.var(r_child))
    return reduction
    
def calculate_leaf_value(self, Y):
    val = np.mean(Y)
    return val
                
def print_tree(self, tree=None, indent=" "):
    if not tree:
        tree = self.root

    if node.value is not None:
        print(tree.value)

    else:
        print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
        print("%sleft:" % (indent), end="")
        self.print_tree(tree.left, indent + indent)
        print("%sright:" % (indent), end="")
        self.print_tree(tree.right, indent + indent)
    
# function to train the tree
def fit(self, X, Y):
    dataset = np.concatenate((X, Y), axis=1)
    self.root = self.build_tree(dataset)
     
# function to predict new dataset
def make_prediction(self, x, tree):
    if tree.value!=None: return tree.value
    feature_val = x[tree.feature_index]
    if feature_val<=tree.threshold:
        return self.make_prediction(x, tree.left)
    else:
        return self.make_prediction(x, tree.right)
  
 # function to predict a single data point
def predict(self, X):
    preditions = [self.make_prediction(x, self.root) for x in X]
    return preditions

In [36]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [37]:
from sklearn.tree import DecisionTreeRegressor
regressor=DecisionTreeRegressor()
regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=3)
regressor.fit(X_train,Y_train)
regressor.print_tree()

ValueError: could not convert string to float: 'Saved! (2004)'

In [ ]:
Y_pred = regressor.predict(X_test) 
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(Y_test, Y_pred))